In [75]:
import requests
from sqlalchemy import create_engine, Table, MetaData, exc
from sqlalchemy import text
import pymysql
from sqlalchemy import create_engine
import urllib.parse

In [76]:
# Substitua pela sua chave de API
api_key = 'c90c917f98d91bd0c1fd096a6b77ab58'

# URL para acessar a API do TMDB
url = f"https://api.themoviedb.org/3/movie/popular?api_key={api_key}&language=pt-BR&page=1"

# Fazendo a requisição à API
response = requests.get(url)

# Verificando se a requisição foi bem-sucedida (código 200)
if response.status_code == 200:
    data = response.json()    
    
    if 'results' in data:
        for movie in data['results']:            
            print(movie.keys())
            print('---')
    else:
        print("Erro: O campo 'results' não foi encontrado na resposta da API.")
else:
    print(f"Erro: Não foi possível acessar a API. Código de status: {response.status_code}")


dict_keys(['adult', 'backdrop_path', 'genre_ids', 'id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'release_date', 'title', 'video', 'vote_average', 'vote_count'])
---
dict_keys(['adult', 'backdrop_path', 'genre_ids', 'id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'release_date', 'title', 'video', 'vote_average', 'vote_count'])
---
dict_keys(['adult', 'backdrop_path', 'genre_ids', 'id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'release_date', 'title', 'video', 'vote_average', 'vote_count'])
---
dict_keys(['adult', 'backdrop_path', 'genre_ids', 'id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'release_date', 'title', 'video', 'vote_average', 'vote_count'])
---
dict_keys(['adult', 'backdrop_path', 'genre_ids', 'id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'release_date', 'title', 'video', 'vote_aver

In [78]:
# Substitua pela sua chave de API
api_key = 'c90c917f98d91bd0c1fd096a6b77ab58'
base_url = 'https://api.themoviedb.org/3/movie/popular'

# Conexão com o MySQL
db_user = 'root'
db_password = 'Are@0051'
db_host = 'localhost'
db_name = 'tmdb'

senha_codificada = urllib.parse.quote_plus(db_password)

# Criação da engine de conexão com o MySQL
engine = create_engine(f'mysql+pymysql://{db_user}:{senha_codificada}@{db_host}/{db_name}')

# Função para truncar a tabela antes de cada inserção
def truncate_table():
    with engine.connect() as connection:
        connection.execute(text('TRUNCATE TABLE filmes_populares_TMDB'))

# Função para buscar filmes da API e inserir no MySQL
def fetch_movies():
    truncate_table()  # Truncando a tabela antes de inserir novos dados
    
    for page in range(1, 6):  # Iterando pelas primeiras 5 páginas
        params = {
            'api_key': api_key,
            'language': 'pt-BR',
            'page': page
        }
        
        response = requests.get(base_url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            if 'results' in data:
                for movie in data['results']:
                    titulo = movie.get('title', 'Título não disponível')
                    data_lancamento = movie.get('release_date', None)
                    popularidade = movie.get('popularity', None)
                    sinopse = movie.get('overview', 'Sinopse não disponível')
                    nota = movie.get('vote_average', None)                    
                    genero_ids = movie.get('genre_ids', [])
                    votos = movie.get('vote_count', None)

                    # Tratando o campo genero_ids: Verifique como esse campo deve ser inserido no banco
                    # Transformando a lista de 'genre_ids' em uma string separada por vírgulas
                    genero_ids_str = ','.join(map(str, genero_ids))

                    # Inserindo os dados na tabela MySQL
                    try:
                        with engine.connect() as connection:
                            connection.execute(
                                text("""
                                INSERT INTO filmes_populares_TMDB (titulo, data_lancamento, popularidade, sinopse, nota, genero_ids, votos)
                                VALUES (:titulo, :data_lancamento, :popularidade, :sinopse, :nota, :genero_ids, :votos)
                                """),
                                {
                                    'titulo': titulo,
                                    'data_lancamento': data_lancamento,
                                    'popularidade': popularidade,
                                    'sinopse': sinopse,
                                    'nota': nota,                                    
                                    'genero_ids': genero_ids_str,  # String formatada corretamente
                                    'votos': votos
                                }
                            )
                    except Exception as e:
                        print(f"Erro ao inserir no banco de dados: {e}")
        else:
            print(f"Erro ao acessar API: {response.status_code}")

# Rodando o script
fetch_movies()